In [1]:
from google.colab import drive
drive.mount('/content/drive'  , force_remount=True
)

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

all_players_sorted_1 = pd.read_pickle("all_players_sorted_1.pkl")
all_players_sorted_2 = pd.read_pickle("all_players_sorted_2.pkl")
all_players_sorted_3 = pd.read_pickle("all_players_sorted_3.pkl")
all_players_sorted_4 = pd.read_pickle("all_players_sorted_4.pkl")
all_players_sorted_5 = pd.read_pickle("all_players_sorted_5.pkl")
all_players_sorted_6 = pd.read_pickle("all_players_sorted_6.pkl")
all_players_sorted_7 = pd.read_pickle("all_players_sorted_7.pkl")
all_players_sorted_8 = pd.read_pickle("all_players_sorted_8.pkl")
all_players_sorted_9 = pd.read_pickle("all_players_sorted_9.pkl")

df_all = pd.read_pickle("df_all.pkl")

games = pd.read_csv("games.csv")
players = pd.read_csv("players.csv")
plays = pd.read_csv("plays.csv")
tackles = pd.read_csv("tackles.csv")

In [ ]:
shap_values_wks_7_9 = pd.read_csv("shap_values_7_9.csv")

In [5]:
!pip install nfl-data-py

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.7 MB/s eta 0:00:00
  Created wheel for nfl-data-py: filename=nfl_data_py-0.3.1-py3-none-any.whl size=13187 sha256=a05a0dad30cbffbf39a5929ca9d035837f94107add86f4301447e08197df970a
  Stored in directory: /root/.cache/pip/wheels/ca/2e/d2/4def312ac7c070e1c0002e91ecc6923481a86c3ebd5b2c5413
Successfully built nfl-data-py


In [6]:
import nfl_data_py as nfl

pbp_data = nfl.import_pbp_data(years=[2022])

2022 done.
Downcasting floats.


In [7]:
# additional play-by-play data from nfl-data-py

pbp_df = pbp_data[['old_game_id', 'play_id', 'third_down_failed', 'fourth_down_failed', 'goal_to_go',
                   'tackled_for_loss', 'run_gap', 'run_location', 'epa']]

pbp_df = pbp_df.rename(columns={'old_game_id': 'gameId', 'play_id': 'playId'})

pbp_df['gameId'] = pbp_df['gameId'].astype(int)

In [8]:
wks_1_6 = games[['gameId', 'week']].merge(df_all[['gameId']])
wks_7_9 = wks_1_6[wks_1_6['week'] > 6.5]
wks_1_6 = wks_1_6[wks_1_6['week'] < 6.5]

df_all_1_6 = df_all[df_all['gameId'] < 2022101900]
df_all_7_9 = df_all[df_all['gameId'] > 2022101900]

In [9]:
# SHAP value sums for defenders 1-3
shap_values_wks_7_9['def1_sum'] = shap_values_wks_7_9['player2.0_s'] + shap_values_wks_7_9['player2.0_dist_ball'] + shap_values_wks_7_9['player2.0_ang_ball'] + shap_values_wks_7_9['player2.0_dis_closest_o'] + shap_values_wks_7_9['player2.0_ang_closest_o'] + shap_values_wks_7_9['player2.0_min_dist'] + shap_values_wks_7_9['player2.0_ang_min'] + shap_values_wks_7_9['player2.0_2_min_dist'] + shap_values_wks_7_9['player2.0_2_ang_min']

shap_values_wks_7_9['def2_sum'] = shap_values_wks_7_9['player3.0_s'] + shap_values_wks_7_9['player3.0_dist_ball'] + shap_values_wks_7_9['player3.0_ang_ball'] + shap_values_wks_7_9['player3.0_dis_closest_o'] + shap_values_wks_7_9['player3.0_ang_closest_o'] + shap_values_wks_7_9['player3.0_min_dist'] + shap_values_wks_7_9['player3.0_ang_min'] + shap_values_wks_7_9['player3.0_2_min_dist'] + shap_values_wks_7_9['player3.0_2_ang_min']

shap_values_wks_7_9['def3_sum'] = shap_values_wks_7_9['player4.0_s'] + shap_values_wks_7_9['player4.0_dist_ball'] + shap_values_wks_7_9['player4.0_ang_ball'] + shap_values_wks_7_9['player4.0_dis_closest_o'] + shap_values_wks_7_9['player4.0_ang_closest_o'] + shap_values_wks_7_9['player4.0_min_dist'] + shap_values_wks_7_9['player4.0_ang_min'] + shap_values_wks_7_9['player4.0_2_min_dist'] + shap_values_wks_7_9['player4.0_2_ang_min']

In [10]:
all_players_sorted_7_9 = pd.concat([all_players_sorted_7, all_players_sorted_8, all_players_sorted_9], axis=0)

In [11]:
df_all_7_9 = df_all_7_9.reset_index(drop=True)

In [12]:
df_shap_7_9 = df_all_7_9[['gameId', 'playId', 'frameId', 'frame_since_bc', 'player1.0_y',
                          'actual_dist_from_final']].merge(shap_values_wks_7_9[['def1_sum', 'def2_sum', 'def3_sum']],
                                                                                  left_index=True, right_index=True)

In [13]:
df_shap_7_9_def = df_shap_7_9.merge(all_players_sorted_7_9[all_players_sorted_7_9['rank'] == 2][['gameId', 'playId', 'frameId', 'nflId', 'displayName', 'new_pos']],
      on=['gameId', 'playId', 'frameId'], how='inner').merge(all_players_sorted_7_9[all_players_sorted_7_9['rank'] == 3][['gameId', 'playId', 'frameId', 'nflId', 'displayName', 'new_pos']],
      on=['gameId', 'playId', 'frameId'], how='inner').merge(all_players_sorted_7_9[all_players_sorted_7_9['rank'] == 4][['gameId', 'playId', 'frameId', 'nflId', 'displayName', 'new_pos']],
      on=['gameId', 'playId', 'frameId'], how='inner')

df_shap_7_9_def = df_shap_7_9_def.rename(columns={'nflId_x': 'def1_nflId', 'displayName_x': 'def1_name', 'nflId_y': 'def2_nflId',
                                'displayName_y': 'def2_name', 'nflId': 'def3_nflId', 'displayName': 'def3_name'})

In [14]:
# cutoff play once the frame gets to the point where the ball carrier has reached final distance and doesn't move side to side anymore

def filter_rows(group):
    # Identify if any frame's distance_to_final drops below 0.1
    cutoff_reached = group['actual_dist_from_final'].lt(0).any()
    if cutoff_reached:
        # Find the first frameId where distance_to_final drops below 0.1
        first_cutoff_frame = group[group['actual_dist_from_final'] < 0]['frameId'].iloc[0]
        # Check if any subsequent frames have distance_to_final > 0.5
        subsequent_frames = group[group['frameId'] > first_cutoff_frame]
        if not subsequent_frames['actual_dist_from_final'].gt(0.5).any():
            # Filter out rows after the first cutoff frame (plus 1 additional frame) and keep the last 15 frames
            remaining_frames = group[(group['frameId'] <= (first_cutoff_frame + 1)) ]            # & (group['frameId'] >= (first_cutoff_frame - 13))
            removing_frames = group[(group['frameId'] > first_cutoff_frame)]
            # Check if the difference between max and min of 'player1.0_y' is less than 2
            if removing_frames['player1.0_y'].max() - removing_frames['player1.0_y'].min() < 2:
                return remaining_frames
    return group

In [15]:
df_shap_7_9_def = df_shap_7_9_def.groupby(['gameId', 'playId']).apply(filter_rows).reset_index(drop=True)

In [16]:
# cutoff to the last 15 frames
df_shap_7_9_def = df_shap_7_9_def.groupby(['gameId', 'playId']).apply(lambda x: x.tail(15)).reset_index(drop=True)

In [17]:
# filter out touchdowns and nullified penalties
plays['TD'] = plays['playDescription'].str.contains('TOUCHDOWN', case=False).astype(int)
filtered_plays = plays[(plays['TD'] == 0) & (plays['playNullifiedByPenalty'] == 'N')].reset_index(drop=True)

In [18]:
df_shap_7_9_def_fp = df_shap_7_9_def.merge(filtered_plays[['gameId', 'playId']], on=['gameId', 'playId'], how='inner')

df_shap_7_9_def_fp['def1_sum_neg_transform'] = np.where(df_shap_7_9_def_fp['def1_sum'] < 0, df_shap_7_9_def_fp['def1_sum'] * -1, 0)
df_shap_7_9_def_fp['def2_sum_neg_transform'] = np.where(df_shap_7_9_def_fp['def2_sum'] < 0, df_shap_7_9_def_fp['def2_sum'] * -1, 0)
df_shap_7_9_def_fp['def3_sum_neg_transform'] = np.where(df_shap_7_9_def_fp['def3_sum'] < 0, df_shap_7_9_def_fp['def3_sum'] * -1, 0)

In [19]:
a = df_shap_7_9_def_fp.groupby(['gameId', 'playId', 'def1_nflId', 'def1_name'])['def1_sum_neg_transform'].sum()
b = df_shap_7_9_def_fp.groupby(['gameId', 'playId', 'def2_nflId', 'def2_name'])['def2_sum_neg_transform'].sum()
c = df_shap_7_9_def_fp.groupby(['gameId', 'playId', 'def3_nflId', 'def3_name'])['def3_sum_neg_transform'].sum()

a = pd.DataFrame(a).reset_index()
b = pd.DataFrame(b).reset_index()
c = pd.DataFrame(c).reset_index()

In [20]:
d = df_shap_7_9_def_fp.groupby(['gameId', 'playId'])['def1_sum_neg_transform'].sum()
e = df_shap_7_9_def_fp.groupby(['gameId', 'playId'])['def2_sum_neg_transform'].sum()
f = df_shap_7_9_def_fp.groupby(['gameId', 'playId'])['def3_sum_neg_transform'].sum()

d = pd.DataFrame(d).reset_index()
e = pd.DataFrame(e).reset_index()
f = pd.DataFrame(f).reset_index()

d = d.rename(columns={'def1_sum_neg_transform': 'def1_total'})
e = e.rename(columns={'def2_sum_neg_transform': 'def2_total'})
f = f.rename(columns={'def3_sum_neg_transform': 'def3_total'})

In [21]:
df_shap_7_9_def_group = a.merge(b, left_on=['gameId', 'playId', 'def1_nflId', 'def1_name'],
                            right_on=['gameId', 'playId', 'def2_nflId', 'def2_name'], how='left').drop(columns=['def2_nflId', 'def2_name'], axis=1).merge(c,
                            left_on=['gameId', 'playId', 'def1_nflId', 'def1_name'], right_on=['gameId', 'playId', 'def3_nflId', 'def3_name'],
                            how='left').drop(columns=['def3_nflId', 'def3_name'], axis=1).merge(d, on=['gameId', 'playId'], how='inner').merge(e,
                            on=['gameId', 'playId'], how='inner').merge(f, on=['gameId', 'playId'], how='inner')

In [22]:
df_shap_7_9_def_group['def1_sum_neg_transform'] = df_shap_7_9_def_group['def1_sum_neg_transform'].fillna(0)
df_shap_7_9_def_group['def2_sum_neg_transform'] = df_shap_7_9_def_group['def2_sum_neg_transform'].fillna(0)
df_shap_7_9_def_group['def3_sum_neg_transform'] = df_shap_7_9_def_group['def3_sum_neg_transform'].fillna(0)

In [23]:
df_shap_7_9_def_group['def_sum'] = df_shap_7_9_def_group['def1_sum_neg_transform'] + df_shap_7_9_def_group['def2_sum_neg_transform'] + df_shap_7_9_def_group['def3_sum_neg_transform']
df_shap_7_9_def_group['play_total'] = df_shap_7_9_def_group['def1_total'] + df_shap_7_9_def_group['def2_total'] + df_shap_7_9_def_group['def3_total']

In [24]:
df_shap_7_9_def_group['prop'] = df_shap_7_9_def_group['def_sum'] / df_shap_7_9_def_group['play_total']
df_shap_7_9_def_group['large_enough'] = np.where(df_shap_7_9_def_group['prop'] > 0.1,1,0)

In [25]:
df_shap_7_9_def_group_over_10pct = df_shap_7_9_def_group[df_shap_7_9_def_group['large_enough'] == 1].reset_index(drop=True)

In [26]:
new_sum = df_shap_7_9_def_group_over_10pct.groupby(['gameId', 'playId'])['def_sum'].sum().reset_index()
new_sum = new_sum.rename(columns={'def_sum': 'new_total_sum'})

In [27]:
df_shap_7_9_def_group_over_10pct = df_shap_7_9_def_group_over_10pct.merge(new_sum, on=['gameId', 'playId'], how='inner')

In [28]:
df_shap_7_9_def_group_over_10pct['new_prop'] = df_shap_7_9_def_group_over_10pct['def_sum'] / df_shap_7_9_def_group_over_10pct['new_total_sum']

In [29]:
df_shap_final = df_shap_7_9_def_group_over_10pct[['gameId', 'playId', 'def1_nflId', 'def1_name', 'new_prop']]

In [30]:
pbp_df['gameId'] = pbp_df['gameId'].astype(int)

In [31]:
df_shap_final_pbp = df_shap_final.merge(pbp_df, on=['gameId', 'playId'], how='inner')

In [32]:
df_shap_final_pbp

,gameId,playId,def1_nflId,def1_name,new_prop,third_down_failed,fourth_down_failed,goal_to_go,tackled_for_loss,run_gap,run_location,epa
0,2022102000,80,44848.0,Budda Baker,0.679478,0.0,0.0,0.0,0.0,None,middle,1.318386
1,2022102000,80,49410.0,Jalen Thompson,0.320522,0.0,0.0,0.0,0.0,None,middle,1.318386
2,2022102000,101,52539.0,Rashard Lawrence,0.641223,0.0,0.0,0.0,0.0,None,middle,-0.152057
3,2022102000,101,53445.0,Zaven Collins,0.358777,0.0,0.0,0.0,0.0,None,middle,-0.152057
4,2022102000,201,40017.0,Tyrann Mathieu,0.511015,0.0,0.0,0.0,0.0,None,middle,0.279343
...,...,...,...,...,...,...,...,...,...,...,...,...
3317,2022110700,3707,44851.0,Marcus Maye,0.294283,0.0,0.0,0.0,0.0,end,right,-0.149736
3318,2022110700,3707,48027.0,Kaden Elliss,0.705717,0.0,0.0,0.0,0.0,end,right,-0.149736
3319,2022110700,3740,48537.0,Carl Granderson,1.000000,1.0,0.0,0.0,1.0,end,left,-1.144850
3320,2022110700,3787,52578.0,Broderick Washington,0.552720,0.0,0.0,0.0,0.0,guard,right,-0.099311


In [33]:
# df_shap_final_pbp.to_csv('df_shap_final_pbp.csv')